# Part 1: Getting started

In [1]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)
import os
os.environ['PATH'] = '/opt/Xilinx/Vivado/2022.1/bin:' + os.environ['PATH']
tf.config.threading.set_inter_op_parallelism_threads(8)
os.environ['OMP_NUM_THREADS'] = '1'

import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from callbacks import all_callbacks

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
import tensorflow.compat.v1 as tf1

2023-07-22 05:35:01.490633: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-22 05:35:01.597256: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Fetch the arrhythmia dataset

In [2]:
model = tf.keras.models.load_model("Arrythmia_model")

2023-07-22 05:35:03.950655: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df = pd.read_csv('./arrhythmia.csv', sep = ',')
print (np.shape(df))
df

(452, 280)


,age,sex,height,weight,QRSduration,PRinterval,Q-Tinterval,Tinterval,Pinterval,QRS,...,chV6_QwaveAmp,chV6_RwaveAmp,chV6_SwaveAmp,chV6_RPwaveAmp,chV6_SPwaveAmp,chV6_PwaveAmp,chV6_TwaveAmp,chV6_QRSA,chV6_QRSTA,Y
0,75,0,190,80,91,193,371,174,121,-16,...,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,8
1,56,1,165,64,81,174,401,149,39,25,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,6
2,54,0,172,95,138,163,386,185,102,96,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,10
3,55,0,175,94,100,202,380,179,143,28,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,1
4,75,0,190,80,88,181,360,177,103,-16,...,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,53,1,160,70,80,199,382,154,117,-37,...,0.0,4.3,-5.0,0.0,0.0,0.7,0.6,-4.4,-0.5,1
448,37,0,190,85,100,137,361,201,73,86,...,0.0,15.6,-1.6,0.0,0.0,0.4,2.4,38.0,62.4,10
449,36,0,166,68,108,176,365,194,116,-85,...,0.0,16.3,-28.6,0.0,0.0,1.5,1.0,-44.2,-33.2,2
450,32,1,155,55,93,106,386,218,63,54,...,-0.4,12.0,-0.7,0.0,0.0,0.5,2.4,25.0,46.6,1


In [4]:
df=df.dropna(axis=1)

In [5]:
X = df.drop('Y', axis = 1).values
y = df.Y
print(y)

0       8
1       6
2      10
3       1
4       7
       ..
447     1
448    10
449     2
450     1
451     1
Name: Y, Length: 452, dtype: int64


In [6]:
X.shape

(452, 274)

In [7]:
y[1:]

1       6
2      10
3       1
4       7
5      14
       ..
447     1
448    10
449     2
450     1
451     1
Name: Y, Length: 451, dtype: int64

In [8]:
le = LabelEncoder()
print(y)
y = le.fit_transform(y)
print(le.classes_)
y = to_categorical(y, 16)
print(y)

0       8
1       6
2      10
3       1
4       7
       ..
447     1
448    10
449     2
450     1
451     1
Name: Y, Length: 452, dtype: int64
[ 1  2  3  4  5  6  7  8  9 10 14 15 16]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [9]:
print(y[1])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [10]:
print(X)
print(y)

[[ 75.    0.  190.  ...   2.9  23.3  49.4]
 [ 56.    1.  165.  ...   2.1  20.4  38.8]
 [ 54.    0.  172.  ...   3.4  12.3  49. ]
 ...
 [ 36.    0.  166.  ...   1.  -44.2 -33.2]
 [ 32.    1.  155.  ...   2.4  25.   46.6]
 [ 78.    1.  160.  ...   1.6  21.3  32.8]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [12]:
X_test

array([[ 51. ,   0. , 170. , ...,   2.6,  23.7,  47.6],
       [ 45. ,   0. , 175. , ...,   1. ,  10.5,  17.9],
       [ 31. ,   1. , 165. , ...,   0.9,   9.6,  14.4],
       ...,
       [ 35. ,   1. , 155. , ...,   0.8,  11.5,  15.5],
       [ 34. ,   0. , 170. , ...,   2.6,  44. ,  68.4],
       [ 49. ,   1. , 162. , ...,   0.5,  15.8,  19.8]])

In [13]:
ls=np.argmax(Y_test,axis=1)
print(ls)
#np.savetxt('output_classes.dat',ls, fmt='%d')

[ 0  5  9 12  5  0  0  0  2  5  0  0  0  8  0  5 10 12  0  1  1  0  3  0
  2  2  0  0  9 12  9  1  9  0  0  2  9  0  0  0  9  0  5  0 11  9  5  0
 12  0  1  0  0  9  0  9  0  0  0  0  9  7  0  0  9  0  0  0  9  0  2  0
 12  0  1  1  0  9  9 12  0  0  9  0  4  0  0  9  4  0  1  0  0  1  0  2
  0  5  0  0  3 12  9  0  4  5  0  0  5  0 10  3  1  0  0  0  9  0  0  1
  0  0  0  4  0  0  9  0  0  2  5  1  0  0 10  0]


In [14]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
# for i in range(0,len(X_train)):
#     X_train[i] = [int(x*64)/64 for x in X_train[i]]
# for i in range(0,len(X_test)):
#     X_test[i] = [int(x*64)/64 for x in X_test[i]]

In [16]:
print(X_test)

[[0.61445783 0.         0.0962963  ... 0.71666667 0.51052632 0.54191818]
 [0.54216867 0.         0.1037037  ... 0.58333333 0.4112782  0.342723  ]
 [0.37349398 1.         0.08888889 ... 0.575      0.40451128 0.31924883]
 ...
 [0.42168675 1.         0.07407407 ... 0.56666667 0.41879699 0.32662643]
 [0.40963855 0.         0.0962963  ... 0.71666667 0.66315789 0.68142186]
 [0.59036145 1.         0.08444444 ... 0.54166667 0.45112782 0.35546613]]


In [17]:
print("Shape of X_train: ",X_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_train: ",Y_train.shape)
print("Shape of y_test",Y_test.shape)

Shape of X_train:  (316, 274)
Shape of X_test:  (136, 274)
Shape of y_train:  (316, 16)
Shape of y_test (136, 16)


In [18]:
# model.evaluate(X_test,Y_test)
# w1=model.get_weights()

In [19]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

import tensorflow as tf
import os
import pandas as pd
import seaborn as sb

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from callbacks import all_callbacks
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu,quantized_po2
import tensorflow.compat.v1 as tf1
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from qkeras.utils import model_save_quantized_weights
from sklearn.preprocessing import MinMaxScaler

In [20]:
model=Sequential()

model.add(QDense(8, input_shape=(274,), name='fc1', kernel_quantizer=quantized_bits(8,1,alpha=1,use_stochastic_rounding=True),bias_quantizer=quantized_bits(8,0,alpha=1),
                kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)   ))
model.add(QActivation(activation=quantized_relu(8,1,use_stochastic_rounding=False), name='relu1'))
model.add(QDense(16, name='output',
                kernel_quantizer=quantized_bits(8,1,alpha=1,use_stochastic_rounding=True), bias_quantizer=quantized_bits(8,0,alpha=1),
                kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001 ) ))
model.add(Activation(activation='softmax', name='softmax'))
#model.set_weights(w1)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [52]:
adam = Adam(lr=0.001)

model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
callbacks= all_callbacks( outputDir = 'Arrythmia_classification_prune')

model.fit(X_train, Y_train, batch_size=1,
          epochs=4,validation_split=0.2, verbose=1, shuffle=True,
          callbacks = callbacks.callbacks);
model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
model_save_quantized_weights(model, "test_weights")

Epoch 1/4
248/252 [============================>.] - ETA: 0s - loss: 0.8171 - accuracy: 0.7581
***callbacks***
saving losses to Arrythmia_classification_prune/losses.log

Epoch 1: val_loss improved from inf to 1.48336, saving model to Arrythmia_classification_prune/KERAS_check_best_model.h5

Epoch 1: val_loss improved from inf to 1.48336, saving model to Arrythmia_classification_prune/KERAS_check_best_model_weights.h5

Epoch 1: saving model to Arrythmia_classification_prune/KERAS_check_model_last.h5

Epoch 1: saving model to Arrythmia_classification_prune/KERAS_check_model_last_weights.h5

***callbacks end***

252/252 [==============================] - 2s 4ms/step - loss: 0.8366 - accuracy: 0.7500 - val_loss: 1.4834 - val_accuracy: 0.6094 - lr: 0.0010
Epoch 2/4
234/252 [==========================>...] - ETA: 0s - loss: 0.7605 - accuracy: 0.8120
***callbacks***
saving losses to Arrythmia_classification_prune/losses.log

Epoch 2: val_loss did not improve from 1.48336

Epoch 2: val_loss d

{'fc1': {'weights': [array([[-0.015625, -0.046875,  0.046875, ...,  0.140625,  0.015625,
            0.      ],
          [ 0.296875, -0.234375,  0.0625  , ...,  0.      ,  0.03125 ,
            0.      ],
          [-0.0625  ,  0.203125, -0.078125, ..., -0.15625 , -0.03125 ,
            0.      ],
          ...,
          [-0.21875 ,  0.015625,  0.125   , ..., -0.1875  , -0.015625,
            0.      ],
          [ 0.03125 , -0.03125 ,  0.015625, ...,  0.1875  ,  0.03125 ,
            0.      ],
          [-0.078125, -0.03125 ,  0.140625, ..., -0.046875,  0.0625  ,
            0.      ]], dtype=float32),
   array([-0.015625 ,  0.0234375, -0.015625 , -0.0546875, -0.0078125,
           0.046875 ,  0.078125 ,  0.       ], dtype=float32)]},
 'output': {'weights': [array([[ 0.265625,  0.8125  , -2.      , -1.25    , -1.125   ,  0.484375,
           -0.21875 , -0.421875, -0.546875, -1.0625  , -0.03125 , -0.59375 ,
           -1.15625 , -0.015625, -0.03125 , -0.015625],
          [ 0.484375

In [62]:
#model = tf.keras.models.load_model("arrythmia")
accuracy=model.evaluate(X_test,Y_test)
#model.save("arrythmia")

5/5 [==============================] - 0s 3ms/step - loss: 1.5762 - accuracy: 0.6176


INFO:tensorflow:Assets written to: arrythmia/assets


INFO:tensorflow:Assets written to: arrythmia/assets


In [ ]:
model.get_weights()

In [54]:
import hls4ml
import plotting

config = hls4ml.utils.config_from_keras_model(model, granularity='name')
config['LayerName']['softmax']['exp_table_t'] = 'ap_fixed<18,8>'
config['LayerName']['softmax']['inv_table_t'] = 'ap_fixed<18,4>'
print("-----------------------------------")
plotting.print_dict(config)
print("-----------------------------------")
hls_model = hls4ml.converters.convert_from_keras_model(
    model, hls_config=config, output_dir='ARRYTHMIA/hls4ml_prj', part='xc7z007s-clg225-2'
)
hls_model.compile()

y_qkeras = model.predict(np.ascontiguousarray(X_test))
y_hls = hls_model.predict(np.ascontiguousarray(X_test))

/home/edge/miniconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/hls4ml/converters/__init__.py:24: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


None
Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: InputLayer, input shapes: [[None, 274]], output shape: [None, 274]
Layer name: fc1, layer type: QDense, input shapes: [[None, 274]], output shape: [None, 8]
Layer name: relu1, layer type: Activation, input shapes: [[None, 8]], output shape: [None, 8]
Layer name: output, layer type: QDense, input shapes: [[None, 8]], output shape: [None, 16]
Layer name: softmax, layer type: Softmax, input shapes: [[None, 16]], output shape: [None, 16]
-----------------------------------
Model
  Precision:         fixed<16,6>
  ReuseFactor:       1
  Strategy:          Latency
  BramFactor:        1000000000
  TraceOutput:       False
LayerName
  fc1_input
    Trace:           False
    Precision
      result:        fixed<16,6>
  fc1
    Trace:           False
    Precision
      result:        fixed<16,6>
      weight:        fixed<8,2>
      bias:          fixed<8,1>
      accum:         fixed<16,6>
    ReuseFactor:     1
  fc1_

In [55]:
import estimate as es

In [82]:
from importlib import reload
reload(es)

<module 'estimate' from '/home/edge/Desktop/argykokk/hls4ml-tutorial/networks/arrrythmia/estimate.py'>

In [99]:
w1=model.get_weights()[0]
w2=model.get_weights()[2]
np.save("weights.npy",model.get_weights()[0])
np.save("weights_2.npy",w2)

In [93]:
w=[274*8]*0

In [96]:
w=list(we)

In [97]:
w

[array([-0.015625, -0.046875,  0.046875, -0.265625,  0.      ,  0.140625,
         0.015625,  0.      ], dtype=float32),
 array([ 0.296875, -0.234375,  0.0625  ,  0.125   ,  0.      ,  0.      ,
         0.03125 ,  0.      ], dtype=float32),
 array([-0.0625  ,  0.203125, -0.078125, -0.046875,  0.      , -0.15625 ,
        -0.03125 ,  0.      ], dtype=float32),
 array([ 0.328125,  0.140625,  0.40625 ,  0.125   ,  0.      , -0.03125 ,
         0.      ,  0.      ], dtype=float32),
 array([-0.421875, -0.390625, -0.328125, -0.25    ,  0.      ,  0.078125,
        -0.078125,  0.      ], dtype=float32),
 array([ 0.203125,  0.078125,  0.015625, -0.03125 ,  0.      ,  0.0625  ,
         0.03125 ,  0.      ], dtype=float32),
 array([ 0.359375, -0.046875,  0.203125,  0.046875,  0.      ,  0.328125,
        -0.03125 ,  0.      ], dtype=float32),
 array([ 0.09375 , -0.265625, -0.390625, -0.609375,  0.      ,  0.5     ,
         0.078125,  0.      ], dtype=float32),
 array([ 0.515625,  0.28125 ,  0

In [100]:
param=1
reuse=1

In [101]:
%%time
#input_num, neurons_num, layer_id, model
luts, ffs = es.estimate(274,8,0,model,reuse,param)
luts, ffs = es.estimate(8,16,2,model,reuse,param)

zero weight are:  924
Mul ins = 119 and Max muls = 1268 and Saved muls = 0 Reuse factor = 1
Muxes LUTS: 0
LUT cost1=: 7378 cost2= 8892 bias acc= 112 mult acc17640
LUTs prediction: 34022
Initial muls: 2192 Real muls: 1268 Initial neurons: 8 Tuned neurons: 5
FFs prediction: 32899
zero weight are:  36
Mul ins = 42 and Max muls = 92 and Saved muls = 0 Reuse factor = 1
Muxes LUTS: 0
LUT cost1=: 2604 cost2= 611 bias acc= 224 mult acc1064
LUTs prediction: 4503
Initial muls: 128 Real muls: 92 Initial neurons: 16 Tuned neurons: 12
FFs prediction: 2165
CPU times: user 19.3 ms, sys: 3.95 ms, total: 23.3 ms
Wall time: 26.1 ms


/home/edge/miniconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/edge/miniconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
